In [1]:
# Реализовать модель сети Хопфилда с дискретным состоянием
# и дискретным временем в асинхронном режиме

In [2]:
import numpy as np
from numbers import Number

In [3]:
class Hopfield:

    def __init__(self, images, nu: float = 1) -> None:
        self.size = images.shape[1]
        self.w = np.matrix(np.zeros((self.size, self.size)))
        self.images = images
        self.neg_images = self._get_neg_images(self.images)
        self.nu = nu

    def _get_neg_images(self, images):
        return images * (-1)      

    def msign(self, s: Number, y_t: Number) -> Number:
        if s > 0:
            return 1
        elif s < 0:
            return -1
        else:
            return y_t 

    def train(self):
        for image in self.images:
            # x = np.matrix(image.copy()).T                        
            # x_plus = np.linalg.pinv(x) # pseudo-inverted matrix                      
            # w = x @ x_plus            
            # # print(w)
            # x_t = image.copy()
            # print(w @ x_t)
            # self.w = self.w + (self.nu / self.size) * (x_t - (w @ x_t)) @ x_t.T
            x = np.matrix(image)            
            self.w += (x.T @ x) / len(self.images)                                                                 
        np.fill_diagonal(self.w, 0)
    
    def _update_neuron(self, x, neuron_idx: int) -> Number:
        return self.msign((self.w[neuron_idx] @ x).item(), x[neuron_idx])
    
    def _find_image_num(self, x, images) -> int | None:
        mask = (images == x).all(axis=1)
        search_result = np.where(mask)[0]
        if np.any(search_result):
            return search_result.item()
        return None

    def predict(self, x, max_iters: int = 100):
        state = x.copy()
        relaxation_iters = 0
        for _ in range(max_iters):            
            relaxation_iters += 1
            prev_state = state.copy()
            for i in range(self.size):                
                state[i] = self._update_neuron(state, i)
            if np.array_equal(prev_state, state):
                is_negative = False
                image_num = self._find_image_num(state, self.images)
                if image_num is None:
                    image_num = self._find_image_num(state, self.neg_images)
                    is_negative = True
                return (relaxation_iters, state,
                        image_num, is_negative)            
        return max_iters, state, None, None

In [4]:
def is_negative(image, neg_image) -> bool:
    return(np.array_equal(image, neg_image))

In [5]:
# [-1, 1, 1, 1, -1, -1, -1, 1, 1],
# [1, 1, -1, 1, -1, 1, 1, 1, -1]
alphabet = np.array(
    [
        [-1, 1, -1, 1, 1, 1, 1, -1, 1],
        [-1, 1, 1, 1, -1, -1, -1, 1, 1],
        [1, 1, -1, 1, -1, 1, 1, 1, -1]       
    ]
)
network = Hopfield(alphabet, 4.5)
network.train()

In [8]:
test_image = np.array([1, -1, -1, -1, 1, 1, 1, -1, -1])
r_iters, state, image_idx, is_negative = network.predict(test_image, 100)
if image_idx:
    predicted_img = network.neg_images[image_idx] if is_negative else network.images[image_idx]
else:
    predicted_img = state
print(f'Prediction success? - {True if image_idx is not None else False}')
print(f'Image number: {image_idx}.\nImage: {predicted_img}.\nIters for relax: {r_iters}')
print(f'Is this image a negative? - {is_negative}')

Prediction success? - True
Image number: 1.
Image: [ 1 -1 -1 -1  1  1  1 -1 -1].
Iters for relax: 1
Is this image a negative? - True
